# This is a notebook for querying the ARGO instance for the 1.0.0 Beacon release.

-   You can run each cell individually by pressing "shift + enter".
-   For more information, questions, bugs, please contact us on Slack:
    -   https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [10]:
Token = ""

#### Install the following packages, if you have not already installed them in your environment:

-   pip install requests
-   pip install xarray
-   pip install ipywidgets
-   pip install cartopy
-   pip install h5netcdf
-   pip install netcdf4
-   pip install scipy
-   pip install packaging


#### Import the required packages


In [11]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the ARGO endpoint

Swagger page here: https://beacon-argo.maris.nl/swagger/


In [12]:
responseinfo = requests.get(
    "https://beacon-argo.maris.nl/api/query/available-columns",
    headers={"Authorization": f"Bearer {Token}"},
)
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [13]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("JULD")  # Enter your search term here

Matching columns:
JULD
JULD.long_name
JULD.standard_name
JULD.units
JULD.conventions
JULD.resolution
JULD._FillValue
JULD.axis
JULD_QC
JULD_QC.long_name
JULD_QC.conventions
JULD_QC._FillValue
JULD_LOCATION
JULD_LOCATION.long_name
JULD_LOCATION.units
JULD_LOCATION.conventions
JULD_LOCATION.resolution
JULD_LOCATION._FillValue


#### You can define here your input parameters


In [14]:
parameter = "TEMP"  # column name
mindate = "2010-01-01"  # yyyy-mm-dd
maxdate = "2011-01-01"  # yyyy-mm-dd
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
mindepth = 0
maxdepth = 10

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

-   For more query examples and explanations, you can take a look at https://maris-development.github.io/beacon/.


In [15]:
def query(
    parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth
):
    body = {
        "query_parameters": [
            {
                "column_name": parameter,
                "alias": parameter,
            },
            {
                "column_name": f"{parameter}_QC",
            },
            {"column_name": "JULD", "alias": "time_ISO8601 [yyyy-MM-ddTHH:mm:ss.SSS]"},
            {"column_name": "PRES", "alias": "Pressure [dbar]"},
            {"column_name": "PRES_QC"},
            {"column_name": "LONGITUDE"},
            {"column_name": "LATITUDE"},
            {
                "function": "concat",
                "args": ["PLATFORM_NUMBER", "CYCLE_NUMBER"],
                "alias": "COMMON_ODV_TAG",
            },
        ],
        "filters": [
            {
                "for_query_parameter": "time_ISO8601 [yyyy-MM-ddTHH:mm:ss.SSS]",
                "min": f"{mindate}T00:00:00",
                "max": f"{maxdate}T00:00:00",
            },
            {
                "for_query_parameter": "Pressure [dbar]",
                "min": mindepth,
                "max": maxdepth,
            },
            {"for_query_parameter": "LONGITUDE", "min": minlon, "max": maxlon},
            {"for_query_parameter": "LATITUDE", "min": minlat, "max": maxlat},
            {
                "is_not_null": {"for_query_parameter": parameter},
            },
        ],
        "output": {
            "format": {
                "odv": {
                    "longitude_column": {"column_name": "LONGITUDE"},
                    "latitude_column": {"column_name": "LATITUDE"},
                    "time_column": {
                        "column_name": "time_ISO8601 [yyyy-MM-ddTHH:mm:ss.SSS]"
                    },
                    "depth_column": {
                        "column_name": "Pressure [dbar]",
                        "qf_column": "PRES_QC",
                    },
                    "data_columns": [
                        {
                            "column_name": f"{parameter}",
                            "qf_column": f"{parameter}_QC",
                        }
                    ],
                    "metadata_columns": [],
                    "qf_schema": "SEADATANET",
                    "key_column": "COMMON_ODV_TAG",
                    # With this line we are asking the zip file to be gzip compressed
                    "archiving": "zip_deflate",
                }
            }
        },
    }
    return body


query_body = query(
    parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth
)

#### This is the post request that is sent to Beacon with the above specified body.


In [16]:
response = requests.post(
    "https://beacon-argo.maris.nl/api/query",
    json.dumps(query_body),
    headers={"Authorization": f"Bearer {Token}", "Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

In [19]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/ARGO.zip",
    "wb",
).write(response.content)

3469817